In [1]:
import os
import requests
from bs4 import BeautifulSoup
import json
from lxml import etree
import logging
import time
import re
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

## Come installare la cartella con tutti i documenti

1.   Vai su Google Drive.
2.   Trova la cartella condivisa "progettoIngDati".
3.   Fai clic con il pulsante destro del mouse sulla cartella e   
     seleziona :
     - Organizza.
     - Aggiungi Scorciatoia
     - Tutte le posizioni
4.   Scegli "My Drive" come destinazione per il collegamento.

In [67]:
SOURCES_DIR = 'sources/'
EXTRACTION_DIR = 'extraction/'

if not os.path.exists(SOURCES_DIR):
    os.makedirs(f'./{SOURCES_DIR}', exist_ok=True)

if not os.path.exists(EXTRACTION_DIR):
    os.makedirs(f'./{EXTRACTION_DIR}', exist_ok=True)

Cancella la cartella articoli/ (se  serve)

In [11]:
!rm -rf sources/

# DOI Scouting (article id) and HTML Download
Questa sezione esegue una query su arxiv.org cercando articoli che parlano di **LLM multi agent** e attraverso *bs4* ottiene i vari [DOI](https://en.wikipedia.org/wiki/Digital_object_identifier).
Con questi DOI è dunque possibile eseguire richieste specifiche per scaricare i documenti HTML del topic scelto.  

Scarica il file HTML5 da ar5iv

Salva l'articolo in un file


In [14]:
def save_to_file(article, article_id):
    # Crea una cartella per gli articoli se non esiste
    #os.makedirs("articoli", exist_ok=True)
    #file_path = os.path.join("articoli", f"ar5iv_article_{article_id}.html")
    #with open(file_path, "w", encoding="utf-8") as file:
    #    file.write(article.prettify())
    #se hai montato la cartella questo funziona, altrimenti usa la parte commentata qui sopra
    file_path = os.path.join(SOURCES_DIR, f"ar5iv_article_{article_id}.html") #join with articoli_path
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(article.prettify())
    logging.info(f"Articolo {article_id} salvato.")

Cerca articoli simili e prova a scaricarne il contenuto html

Punto di lancio del codice, la query è composta da `url + params`

In [ ]:
# Define the URL for the advanced search on arXiv
url = "https://arxiv.org/search/advanced"

# Define the search parameters
params = {
    "advanced": "",
    "terms-0-term": TOPIC,              # Search term
    "terms-0-operator": "AND",          # Operator
    "terms-0-field": "all",             # Search field (all)
    "classification-computer_science": "y",  # Limit to computer science
    "classification-physics_archives": "all",  # Include all physics archives
    "classification-include_cross_list": "include",  # Include cross-list
    "date-filter_by": "all_dates",      # Filter by date (all dates)
    "date-year": "",                    # No specific year
    "date-from_date": "",               # No start date
    "date-to_date": "",                 # No end date
    "date-date_type": "submitted_date", # Search by submission date
    "abstracts": "hide",                # Show abstracts
    "size": 200,                        # Number of results to display
    "order": "submitted_date"           # Order by submission date (oldest first) #older documents are more likely to have html version
}

doi_retrieved = 0
while doi_retrieved <= 300:
    doi_retrieved += retrieve_article_id(url, params)
    params["start"] = doi_retrieved

logging.info(f"Number of articles retrieved: {len(article_ids)}")

# Data Extraction

In [79]:
# Funzione per salvare i dati estratti in un file JSON
def salva_dati_in_json(dati_estratti, article_id, extraction_path):
    tabelle = dati_estratti

    # Salva i dati in un file JSON
    output_file_path = f"{extraction_path}/{article_id}.json"
    os.makedirs(extraction_path, exist_ok=True)
    with open(output_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(tabelle, json_file, ensure_ascii=False, indent=4)

    #logging.info(f"Dati salvati in {output_file_path}")

In [61]:
def estrai_dati_da_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    parser = etree.HTMLParser()
    tree = etree.fromstring(content, parser)

    tabelle = {}
    table_counter = 0

    figure_with_tables = tree.xpath('//figure[.//table]')

    for figure in figure_with_tables:
        try:
            try:
                table_id = figure.xpath("@id")[0]
            except:
                # cerca l'id al secondo div padre perché a volte è salvato lì
                table_id = figure.xpath('ancestor::div[2]/@id')[0]
            table = figure.xpath('.//table')[0]
            table_counter += 1
            table_key = f"id_table_{table_counter}"

            caption = figure.xpath('.//figcaption//text()')
            caption_text = '' 

            if caption:
                caption_text = ''.join([c.strip() for c in caption]).replace('  ', ' ')

            dati_tabella = []
            rows = table.xpath('.//tr[position()>1]')
            for row in rows:
                cols = row.xpath('.//td')
                dati_row = [etree.tostring(col, encoding='unicode', method='html') for col in cols]
                dati_tabella.append(dati_row)

            references = tree.xpath(f"//p[a/@href = '#{table_id}']")
            references_text = [ref.xpath('string(.)').replace('\n', '').strip() for ref in references]   #elimina /n ma introduce spazi bianchi
            
            note_a_pie_di_pagina = []
            footnotes = figure.xpath(".//span[contains(@id, 'footnote') and contains(@class, 'ltx_text')]")
            for footnote in footnotes:
                logging.info(f"footnote found: {footnote} at table: {table_id}")

                footnote_text = " ".join(footnote.xpath(".//text()")).strip()
                if footnote_text and footnote_text != " ":
                    footnote_text = footnote_text.replace("\n", " ")
                    note_a_pie_di_pagina.append(footnote_text)

            tabelle[table_key] = {
                "caption": caption_text,
                "table": dati_tabella,
                "footnotes": note_a_pie_di_pagina,
                "references": references_text,
            }

        except Exception as e:
            logging.error(f"figure info: {figure.getchildren()}")
            logging.error(f"Error processing figure in {file_path}: {e}")
            logging.error(f"Figure content: {etree.tostring(figure, encoding='unicode', pretty_print=True)}")
        
    return tabelle

In [6]:
# per testare su di un unico file
file_path = os.path.join(SOURCES_DIR, 'ar5iv_article_2406.14952.html')

dati_estratti = estrai_dati_da_file(file_path)
logging.info(dati_estratti)
# Salva i dati in JSON
salva_dati_in_json(dati_estratti, '0', EXTRACTION_DIR)

2024-10-17 01:23:15,481 - INFO - {'id_table_1': {'caption': 'Table 1:Human judgementZH/ENresults of different role-playing agents.', 'table': [['<td class="ltx_td ltx_align_center ltx_border_r ltx_border_t" id="S2.T1.1.1.1.1">\n          GPT-4\n          <sub class="ltx_sub" id="S2.T1.1.1.1.1.1">\n           <span class="ltx_text ltx_font_italic" id="S2.T1.1.1.1.1.1.1">\n            zero_shot\n           </span>\n          </sub>\n         </td>\n         ', '<td class="ltx_td ltx_align_justify ltx_align_top ltx_border_t" id="S2.T1.1.1.1.2">\n          <span class="ltx_inline-block ltx_align_top" id="S2.T1.1.1.1.2.1">\n           <span class="ltx_p" id="S2.T1.1.1.1.2.1.1" style="width:35.6pt;">\n            9.9/\n            <span class="ltx_text ltx_font_bold" id="S2.T1.1.1.1.2.1.1.1">\n             9.8\n            </span>\n           </span>\n          </span>\n         </td>\n         ', '<td class="ltx_td ltx_align_justify ltx_align_top ltx_border_t" id="S2.T1.1.1.1.3">\n         

In [ ]:
# Itera attraverso i file nella cartella 'articoli' e estrae le tabelle, didascalie, note e riferimenti
total_tables = 0
total_captions = 0
total_footnotes = 0
total_references = 0
articolo_counter = 0

#questo é perché nel salvataggio dei file ho creato cartella articoli dentro a sources
for filename in os.listdir(SOURCES_DIR):
    file_path = os.path.join(SOURCES_DIR, filename)
    article_id = filename.split('_')[2].replace('.html', '')  # Estrae l'ID dal nome del file (primo elemento prima del punto)

    articolo_counter += 1  # Incrementa il contatore
    logging.info(f"Estraendo dati dall'articolo {articolo_counter}: {filename}...")

    # Estrai i dati dal file HTML
    dati_estratti = estrai_dati_da_file(file_path)

    # Update statistics counters
    for table_data in dati_estratti.values():
        total_tables += 1
        if table_data["caption"]:
            total_captions += 1
        total_footnotes += len(table_data["footnotes"])
        total_references += len(table_data["references"])

    # Salva i dati in JSON
    salva_dati_in_json(dati_estratti, article_id, EXTRACTION_DIR)
        
# Print statistics
logging.info("\n--- Extraction Statistics ---")
logging.info(f"Total Articles Processed: {articolo_counter}")
logging.info(f"Total Tables Found: {total_tables}")
logging.info(f"Total Captions Found: {total_captions}")
logging.info(f"Total Footnotes Found: {total_footnotes}")
logging.info(f"Total References Found: {total_references}")

# Valutazione

### Numero di tabelle estratte sul numero di tabelle totali

In [168]:
# Assuming SOURCES_DIR is defined
def evaluate():
    filepaths = os.listdir(SOURCES_DIR)
    totale_html = 0
    totale_json = 0
    pattern = re.compile(r'[a-zA-Z]\d+\.T(\d+)')

    # Wrap the loop with tqdm for progress tracking
    for filename in filepaths:
        article_id = filename.split('_')[2].replace('.html', '')
        json_filename = f"{article_id}.json"
        json_filepath = os.path.join(EXTRACTION_DIR, json_filename)

        with open(os.path.join(SOURCES_DIR, filename), 'r', encoding='utf-8') as file, open(json_filepath, 'r', encoding='utf-8') as json_file:
            html_content = file.read()
            soup = BeautifulSoup(html_content, 'html.parser')

            ty_values = set()
            for tag in soup.find_all(id=pattern):
                match = pattern.search(tag['id'])
                if match:
                    ty_values.add(match.group())

            json_data = json.load(json_file)
            tables_in_html = len(ty_values)
            tables_in_json = len(json_data)

            totale_html += tables_in_html
            totale_json += tables_in_json
            if tables_in_json != tables_in_html:
                logging.warning(f"Found {tables_in_json}/{tables_in_html} tables (JSON/HTML) in {filename}")
            else:
                logging.info(f"Found: {tables_in_json}/{tables_in_html} tables (JSON/HTML) in {filename}")

    logging.info(f"Numero totale di tabelle trovate: {totale_json} su {totale_html}")


In [ ]:
evaluate()

Confrontando le tabelle estratte con l'xPath: `//figure[.//table]` con le tabelle trovate nell'html cercando gli `id` contenenti la lettera **T** seguita da un numero, notiamo che la nostra assunzione è errata. Questo perché non sempre le tabelle sono formattate tramite un tag `<table>`, ad esempio nell'articolo 2309.17288 nessuna delle tre tabelle presenti compare in un tag `<table>`. \
\
Abbiamo quindi trovato **1879** tabelle tramite l'xPath su un totale di **1914** tabelle  

Riscrivo l'estrazione con un nuovo xpath più preciso

In [164]:
def estrai_dati_da_file_v2(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    parser = etree.HTMLParser()
    tree = etree.fromstring(content, parser)

    tabelle = {}
    table_counter = 0

    elements = tree.xpath('//*[starts-with(@id, "S") or starts-with(@id, "A")][contains(@id, ".T") and contains(@class, "ltx_table")]')
    for element in elements:
        try:
            table_counter += 1
            table_id = element.attrib['id']
            if element.tag != 'figure':
                element = element.xpath('.//figure')[0]
            table = element.xpath('.//*[contains(@class, "ltx_tabular")]')[0]
            table_key = f"id_table_{table_counter}"
            caption = element.xpath('.//figcaption[contains(@class, "ltx_caption")]/text()')
            caption_text = ''

            if caption:
                caption_text = ''.join([c.strip() for c in caption]).replace('  ', ' ')
            
            rows = table.xpath('.//*[contains(@class, "ltx_tr")]')
            dati_tabella = []
            
            for row in rows:
                dati_tabella.append([etree.tostring(col, encoding='unicode', method='html',pretty_print=True) for col in row.xpath('.//*[contains(@class, "ltx_td")]')])
            
            references = tree.xpath(f"//p[a/@href = '#{table_id}']")
            references_text = [ref.xpath('string(.)').replace('\n', '').strip() for ref in references]   #elimina /n ma introduce spazi bianchi
                
            note_a_pie_di_pagina = []
            footnotes = element.xpath(".//span[contains(@id, 'footnote') and contains(@class, 'ltx_text')]")
            for footnote in footnotes:
                footnote_text = " ".join(footnote.xpath(".//text()")).strip()
                if footnote_text and footnote_text != " ":
                    footnote_text = footnote_text.replace("\n", " ")
                    note_a_pie_di_pagina.append(footnote_text)


            tabelle[table_key] = {
                "caption": caption_text,
                "table": dati_tabella,
                "footnotes": note_a_pie_di_pagina,
                "references": references_text,
                }

        except Exception as e:
            logging.error(f"figure info: {element.getchildren()}")
            logging.error(f"Error processing figure in {file_path}: {e.with_traceback(None)}")
            logging.error(f"Figure content: {etree.tostring(element, encoding='unicode', pretty_print=True)}")

    return tabelle

In [ ]:
# per testare su di un unico file
file_path = os.path.join(SOURCES_DIR, 'ar5iv_article_2402.07401.html')

dati_estratti = estrai_dati_da_file_v2(file_path)
logging.info(dati_estratti)
# Salva i dati in JSON
salva_dati_in_json(dati_estratti, '0', EXTRACTION_DIR)

In [165]:
# Itera attraverso i file nella cartella 'articoli' e estrae le tabelle, didascalie, note e riferimenti
total_tables = 0
total_captions = 0
total_footnotes = 0
total_references = 0
articolo_counter = 0

#questo é perché nel salvataggio dei file ho creato cartella articoli dentro a sources
for filename in os.listdir(SOURCES_DIR):
    file_path = os.path.join(SOURCES_DIR, filename)
    article_id = filename.split('_')[2].replace('.html', '')  # Estrae l'ID dal nome del file (primo elemento prima del punto)

    articolo_counter += 1  # Incrementa il contatore
    logging.info(f"Estraendo dati dall'articolo {articolo_counter}: {filename}...")

    # Estrai i dati dal file HTML
    dati_estratti = estrai_dati_da_file_v2(file_path)

    # Update statistics counters
    for table_data in dati_estratti.values():
        total_tables += 1
        if table_data["caption"]:
            total_captions += 1
        total_footnotes += len(table_data["footnotes"])
        total_references += len(table_data["references"])

    # Salva i dati in JSON
    salva_dati_in_json(dati_estratti, article_id, EXTRACTION_DIR)
        
# Print statistics
logging.info("\n--- Extraction Statistics ---")
logging.info(f"Total Articles Processed: {articolo_counter}")
logging.info(f"Total Tables Found: {total_tables}")
logging.info(f"Total Captions Found: {total_captions}")
logging.info(f"Total Footnotes Found: {total_footnotes}")
logging.info(f"Total References Found: {total_references}")

2024-10-18 12:54:24,367 - INFO - Estraendo dati dall'articolo 1: ar5iv_article_2401.17749.html...
2024-10-18 12:54:24,373 - INFO - Estraendo dati dall'articolo 2: ar5iv_article_2401.02500.html...


2024-10-18 12:54:24,448 - INFO - Estraendo dati dall'articolo 3: ar5iv_article_2305.15021.html...
2024-10-18 12:54:24,494 - INFO - Estraendo dati dall'articolo 4: ar5iv_article_2402.14871.html...
2024-10-18 12:54:24,507 - INFO - Estraendo dati dall'articolo 5: ar5iv_article_2406.06211.html...
2024-10-18 12:54:24,553 - INFO - Estraendo dati dall'articolo 6: ar5iv_article_2406.16273.html...
2024-10-18 12:54:24,600 - INFO - Estraendo dati dall'articolo 7: ar5iv_article_2306.06687.html...
2024-10-18 12:54:24,635 - INFO - Estraendo dati dall'articolo 8: ar5iv_article_2407.10022.html...
2024-10-18 12:54:24,657 - INFO - Estraendo dati dall'articolo 9: ar5iv_article_2311.09510.html...
2024-10-18 12:54:24,674 - INFO - Estraendo dati dall'articolo 10: ar5iv_article_2310.16301.html...
2024-10-18 12:54:24,701 - INFO - Estraendo dati dall'articolo 11: ar5iv_article_2407.17086.html...
2024-10-18 12:54:24,712 - INFO - Estraendo dati dall'articolo 12: ar5iv_article_2401.16107.html...
2024-10-18 12:54:

In [ ]:
evaluate()

Considerazioni sui risultati ottenuti:\
Alcune tabelle non sono trovate mediante xpath, anche se esso è molto più selettivo e preciso del precedente, poiché i documenti HTML sono malformati (es.):
* il documento 2405.15145 che ha la prima tabella separata dal tag `<figure>` 
* il documento 2310.02071 che costruisce una tabella attraverso il tag `<svg>` il quale è renderizzato con errori sul documento.

### Numero di Footnotes, Captions, Paragraphs with references estratti su totale

In [ ]:
# Controlla se qualche caption inizia per 'Figure' , indicando così che è una caption di un'immagine
json_filenames = os.listdir(EXTRACTION_DIR)
for filename in json_filenames:
    with open(os.path.join(EXTRACTION_DIR,filename), 'r') as f:
        json_data = json.load(f)

        for key, value in json_data.items():
            if 'caption' in value:
                if value['caption'].startswith('Figure'):
                    logging.warning(value['caption'])